# テキスト分類タスク

前述の通り、今回は**AG_NEWS**データセットを使用したシンプルなテキスト分類タスクに焦点を当てます。このタスクでは、ニュースの見出しを「世界」「スポーツ」「ビジネス」「科学/技術」の4つのカテゴリのいずれかに分類します。

## データセット

このデータセットは[`torchtext`](https://github.com/pytorch/text)モジュールに組み込まれているため、簡単にアクセスできます。


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

ここで、`train_dataset` と `test_dataset` は、それぞれクラス番号（ラベル）とテキストのペアを返すコレクションを含んでいます。例えば:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

では、データセットから最初の10件の新しい見出しを出力しましょう。


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

データセットはイテレーターであるため、データを複数回使用したい場合はリストに変換する必要があります。


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## トークン化

次に、テキストを**数値**に変換し、それをテンソルとして表現できるようにする必要があります。単語レベルの表現を求める場合、以下の2つの作業が必要です：
* **トークナイザー**を使用してテキストを**トークン**に分割する
* それらのトークンの**語彙**を構築する


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

語彙を使用することで、トークン化された文字列を簡単に数値のセットにエンコードできます。


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Bag of Words テキスト表現

言葉は意味を表すため、時には文中の順序に関係なく、個々の単語を見るだけでテキストの意味を推測できることがあります。例えば、ニュースを分類する際に、*weather* や *snow* といった単語は *天気予報* を示す可能性が高く、*stocks* や *dollar* といった単語は *金融ニュース* に関連すると考えられます。

**Bag of Words** (BoW) ベクトル表現は、最も一般的に使用される伝統的なベクトル表現です。各単語はベクトルのインデックスにリンクされ、ベクトル要素には特定の文書内での単語の出現回数が含まれます。

![Bag of Words ベクトル表現がメモリ内でどのように表現されるかを示す画像。](../../../../../translated_images/ja/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW は、テキスト内の個々の単語に対するすべてのワンホットエンコードされたベクトルの合計として考えることもできます。

以下は、Scikit Learn の Python ライブラリを使用して Bag of Words 表現を生成する例です:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWSデータセットのベクトル表現からbag-of-wordsベクトルを計算するには、次の関数を使用できます:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **注意:** ここでは、デフォルトの語彙サイズを指定するためにグローバル変数 `vocab_size` を使用しています。語彙サイズが非常に大きい場合が多いため、最も頻繁に使用される単語に語彙サイズを制限することができます。`vocab_size` の値を下げて以下のコードを実行し、その影響を確認してください。精度が多少低下することが予想されますが、劇的な変化はなく、性能が向上するはずです。


## BoW分類器のトレーニング

テキストのBag-of-Words表現を構築する方法を学んだので、その上で分類器をトレーニングしてみましょう。まず、トレーニング用のデータセットを変換する必要があります。すべての位置ベクトル表現をBag-of-Words表現に変換する方法です。これを実現するには、標準的なtorchの`DataLoader`に`collate_fn`パラメータとして`bowify`関数を渡します。


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

では、1つの線形層を含むシンプルな分類器ニューラルネットワークを定義しましょう。入力ベクトルのサイズは `vocab_size` に等しく、出力サイズはクラス数（4）に対応します。分類タスクを解いているため、最終的な活性化関数は `LogSoftmax()` です。


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

これから標準的なPyTorchのトレーニングループを定義します。データセットが非常に大きいため、学習目的のために1エポックのみ、場合によっては1エポック未満でトレーニングを行います（`epoch_size`パラメータを指定することでトレーニングを制限できます）。また、トレーニング中に累積されたトレーニング精度を報告します。報告の頻度は`report_freq`パラメータを使用して指定します。


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## バイグラム、トライグラム、Nグラム

バッグオブワーズアプローチの1つの制約は、一部の単語が複数の単語で構成される表現の一部である場合があることです。例えば、「hot dog」という単語は、他の文脈での「hot」や「dog」という単語とは全く異なる意味を持ちます。「hot」と「dog」を常に同じベクトルで表現すると、モデルが混乱する可能性があります。

これに対処するために、**Nグラム表現**が文書分類の手法でよく使用されます。ここでは、各単語、2単語（バイグラム）、または3単語（トライグラム）の頻度が分類器を訓練するための有用な特徴となります。例えば、バイグラム表現では、元の単語に加えて、すべての単語ペアを語彙に追加します。

以下は、Scikit Learnを使用してバイグラムのバッグオブワード表現を生成する方法の例です：


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gramアプローチの主な欠点は、語彙サイズが非常に速く増加し始めることです。実際には、N-gram表現を*埋め込み*などの次元削減技術と組み合わせる必要があります。この点については次のユニットで説明します。

**AG News**データセットでN-gram表現を使用するには、特別なN-gram語彙を構築する必要があります。


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


次のコードを使って分類器を訓練することもできますが、それでは非常にメモリ効率が悪くなります。次のユニットでは、埋め込みを使用してバイグラム分類器を訓練します。

> **Note:** テキスト内で指定された回数以上出現するn-gramのみを残すことができます。これにより、頻度の低いバイグラムが省略され、次元数が大幅に減少します。そのためには、`min_freq`パラメータを高い値に設定し、語彙の長さの変化を観察してください。


## Term Frequency Inverse Document Frequency TF-IDF

BoW表現では、単語の出現頻度は単語そのものに関係なく均等に重み付けされます。しかし、*a* や *in* などの頻繁に使われる単語は、専門用語に比べて分類において重要性が低いことは明らかです。実際、ほとんどのNLPタスクでは、ある単語が他の単語よりも重要である場合があります。

**TF-IDF**は、**term frequency–inverse document frequency（単語頻度–逆文書頻度）**の略です。これはBoWの変形で、単語が文書に出現するかどうかを示す二値の0/1ではなく、コーパス内での単語の出現頻度に関連する浮動小数点値を使用します。

より正式には、文書$j$内の単語$i$の重み$w_{ij}$は以下のように定義されます：
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
ここで、
* $tf_{ij}$は文書$j$内で単語$i$が出現する回数、つまり以前に見たBoW値
* $N$はコレクション内の文書数
* $df_i$はコレクション全体で単語$i$を含む文書数

TF-IDF値$w_{ij}$は、単語が文書内で出現する回数に比例して増加し、コーパス内でその単語を含む文書数によって調整されます。これにより、ある単語が他の単語よりも頻繁に出現する事実を補正することができます。例えば、単語がコレクション内の*すべて*の文書に出現する場合、$df_i=N$となり、$w_{ij}=0$となります。この場合、その単語は完全に無視されます。

Scikit Learnを使用すれば、簡単にテキストのTF-IDFベクトル化を作成できます：


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## 結論

しかし、TF-IDF表現は異なる単語に頻度の重みを与えるものの、意味や順序を表現することはできません。有名な言語学者J.R.ファースが1935年に述べたように、「単語の完全な意味は常に文脈に依存しており、文脈を無視した意味の研究は真剣に受け止められるべきではない」のです。このコースの後半では、言語モデルを使用してテキストから文脈情報を捉える方法を学びます。



---

**免責事項**:  
この文書は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があることをご承知ください。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤認について、当方は一切の責任を負いません。
